In [1]:
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
import scipy.optimize as opt
%matplotlib inline

In [2]:
base_dir = "/Users/Jackie/Dropbox/Work/machine_learning/hw/machine-learning-ex4/ex4"

In [3]:
data = loadmat(os.path.join(base_dir, 'ex4data1'))
data_weights = loadmat(os.path.join(base_dir, 'ex4weights'))

In [4]:
Theta1 = data_weights['Theta1']
Theta2 = data_weights['Theta2']
print(Theta1.shape, Theta2.shape)

input_layer_size = 400
hidden_layer_size = 25
num_labels = 10
nn_parms = np.concatenate([Theta1.flatten(), Theta2.flatten()])

(25, 401) (10, 26)


In [21]:
X = data['X']
y = data['y']
y = y[:, 0]
m = X.shape[0]
print(X.shape)
print(y.shape)

(5000, 400)
(5000,)


In [6]:
def sigmoid(z):    
    res = 1 / (1+ np.exp(-z))
    return res 

In [7]:
def forward_propagate(X, theta1, theta2): 
    """
        feed forward propagation
        X is orignal data with m obs, and n features 
        
        theta output_size * (input_size + 1)
        X*theta.T --> next layer 
        
    """
    # add constant feature    
    a1 = np.insert(X, 0, values=np.ones(X.shape[0]), axis=1)    
    z2 = a1.dot(theta1.T)
    a2 = sigmoid(z2) #
    
   # add constant feature     
    a2 = np.insert(a2, 0, values=np.ones(a2.shape[0]), axis=1)    
    z3 =a2.dot(theta2.T)
    a3 = sigmoid(z3)
    
    return a1, z2, a2, z3, a3

In [8]:
def nnCost(params, input_size, hidden_size, num_labels, X, y, penalty=0):
    """
        params are 1D array including theta1 and theta2
        input_size: size of 1st layer
        hidden_size: size of second layer
        num_labels: size of third/last layer
        X: m *input_size 2D array (m observations)
        y: 1D array
    """
    # number of obs 
    m = X.shape[0]    
    
    # convert y to binary 2d array with num_labels cols 
    y_list = []
    for i in range(1, num_labels+1):
        y_list.append(y == i ) 
    y = np.vstack(y_list).T
    
    
    # theta is in shape (output_size , input_size +1)
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1)))
    theta2 = np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1)))
        
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)    
    theta_sum = np.sum(np.power(theta1[:, 1:],2)) + np.sum(np.power(theta2[:, 1:],2)) 
        
    
    # compute the cost        
    d = - (y*np.log(h) + (1-y)*np.log(1-h)) #piecewise
    J = np.sum(d) / m  + theta_sum * penalty /(2*m)           
    return J    
        

In [22]:
J = nnCost(nn_parms, input_layer_size, hidden_layer_size, num_labels, X, y, penalty=0)
print("cost function with penalty 0:", J)
J = nnCost(nn_parms, input_layer_size, hidden_layer_size, num_labels, X, y, penalty=1)
print("cost function with penalty 1:", J)

cost function with penalty 0: 0.2876291651613189
cost function with penalty 1: 0.38376985909092365


In [11]:
def randInitializeWeights(nRow, nCol):
    e_init = 0.12
    theta = np.random.uniform(e_init, -e_init, (nRow, nCol))    
    return theta    

In [12]:
def sigmoidGradient(Z):
    a = 1/(1+ np.exp(-Z))
    g = a * (1-a)
    return g 

print(sigmoidGradient(np.array([-1, -0.5, 0, 0.5, 1])) )

def backprop(params, input_size, hidden_size, num_labels, X, y, penalty=0):
    """
        params are 1D array after flattening theta1 and theta2
        input_size: size of 1st layer
        hidden_size: size of second layer
        num_labels: size of third/last layer
        X: m *input_size 2D array (m observations)
        y: 1D array
    """
    # number of obs 
    m = X.shape[0]    
    
    # convert y to binary 2d array with num_labels cols 
    y_list = []
    for i in range(1, num_labels+1):
        y_list.append(y == i ) 
    y = np.vstack(y_list).T
    
    
    # theta is in shape (output_size , input_size +1)
    # reshape the parameter array into parameter matrices for each layer
    # theta1 (25, 401)
    # theta2 (10, 26)
    theta1 = np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1)))
    theta2 = np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1)))
        
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)    
    theta_sum = np.sum(np.power(theta1[:, 1:],2)) + np.sum(np.power(theta2[:, 1:],2)) 
    
    
    # perform backpropagation 
    # d3, d2 are errors in layer 3, layer2
    d3 = h - y  #(5000, 10)            
    z2 = np.insert(z2, 0, values=np.ones(z2.shape[0]), axis=1) # (5000, 26)       
    d2 = d3.dot(theta2) * sigmoidGradient(z2) # (5000, 26)
    
    # complicated math proof --> grad_l(i,j) = a_l(j) * d_l+1(i) 
    grad2 = d3.T.dot(a2) /m  #  (10, 5000) * (5000, 25)  -->  (10, 26)
    
    grad1 = d2[:, 1:].T.dot(a1)/m  # (25, 5000) * (5000, 401) -->  (25, 401)    
    
        
    
    #add the gradient regularization term 
    grad1[:, 1:] = grad1[:, 1:] + theta1[:, 1:] * penalty / m 
    grad2[:, 1:] = grad2[:, 1:] + theta2[:, 1:] * penalty / m 
    
    #flatten grads 
    grad = np.concatenate((np.ravel(grad1), np.ravel(grad2)))
    # compute the cost        
    d = - (y*np.log(h) + (1-y)*np.log(1-h)) #piecewise
    J = np.sum(d) / m  + theta_sum * penalty /(2*m)           
    return J , grad   
        

[0.19661193 0.23500371 0.25       0.23500371 0.19661193]


In [13]:
def computeNumericalGradient(func, params, *args):
    numgrad = np.zeros(params.shape)
    perturb = np.zeros(params.shape)
    e = 1e-4
    for i in range(len(params)):
        perturb[i] = e        
        loss1, _ = func(params+perturb, *args)
        loss2, _ = func(params-perturb, *args)
        numgrad[i] = (loss1 - loss2)/ (2*e)
        perturb[i] = 0
        
    return numgrad        
        

In [14]:
J, grad = backprop(nn_parms, input_layer_size, hidden_layer_size, num_labels, X, y, penalty=1)
print(grad)

[ 6.18712766e-05 -2.11248326e-12  4.38829369e-13 ...  4.70513145e-05
 -5.01718610e-04  5.07825789e-04]


In [15]:
def checkNNGradients(penalty):
    """gradient checking"""
    input_layer_size = 3
    hidden_layer_size = 5
    num_labels = 3
    m = 5
    theta1 = randInitializeWeights(hidden_layer_size, input_layer_size+1)
    theta2 = randInitializeWeights(num_labels, hidden_layer_size+1)    
    X  = randInitializeWeights(m, input_layer_size)
    y = 1 + np.mod(range(m),num_labels)
    nn_parms = np.concatenate((np.ravel(theta1), np.ravel(theta2)))
    print(nn_parms.shape)
    J, grad = backprop(nn_parms, input_layer_size, hidden_layer_size, num_labels, X, y, penalty)    
    numgrad = computeNumericalGradient(backprop, nn_parms, input_layer_size, hidden_layer_size, num_labels, X, y, penalty)
    print(grad)
    print(numgrad)
    
    print(np.linalg.norm(grad-numgrad) / np.linalg.norm(grad+numgrad))

checkNNGradients(1)

(38,)
[-0.00807759 -0.02267003 -0.01382068 -0.01715889  0.00365887  0.00929486
 -0.01247593 -0.00150642  0.00203073 -0.01890614 -0.02216543 -0.00101812
  0.00965727 -0.00233986  0.01697357 -0.00473532  0.00467024 -0.00236324
 -0.00628784  0.00828812  0.05941649  0.03720971  0.00883356  0.01233656
  0.02307268  0.02862346  0.10501174  0.02919758  0.07814001  0.0302043
  0.07560734  0.0680216   0.34333312  0.15342453  0.18092263  0.18058884
  0.1909293   0.17758094]
[-0.00807759 -0.02267003 -0.01382068 -0.01715889  0.00365887  0.00929486
 -0.01247593 -0.00150642  0.00203073 -0.01890614 -0.02216543 -0.00101812
  0.00965727 -0.00233986  0.01697357 -0.00473532  0.00467024 -0.00236324
 -0.00628784  0.00828812  0.05941649  0.03720971  0.00883356  0.01233656
  0.02307268  0.02862346  0.10501174  0.02919758  0.07814001  0.0302043
  0.07560734  0.0680216   0.34333312  0.15342453  0.18092263  0.18058884
  0.1909293   0.17758094]
4.8035109282725497e-11


In [25]:
import scipy.optimize as opt

t1 = randInitializeWeights(hidden_layer_size, input_layer_size+1)
t2 = randInitializeWeights(num_labels, hidden_layer_size+1)
params = np.concatenate([t1.flatten(), t2.flatten()])
penalty = 1
# minimize the objective function
fmin = opt.minimize(fun=backprop, x0=params, args=(input_layer_size, hidden_layer_size, num_labels, X, y, penalty), 
                method='TNC', jac=True, options={'maxiter': 50})
print(fmin)


     fun: 0.8291747208654137
     jac: array([ 4.89373134e-03,  1.68054436e-05,  9.75643588e-07, ...,
       -6.36766995e-03, -7.30806095e-04, -7.92794799e-03])
 message: 'Max. number of function evaluations reached'
    nfev: 50
     nit: 10
  status: 3
 success: False
       x: array([ 0.15632415,  0.08402722,  0.00487822, ..., -0.09090493,
       -3.4259673 , -0.78276053])


In [18]:
theta_prime_1 = np.reshape(fmin.x[:hidden_layer_size * (input_layer_size + 1)], (hidden_layer_size, (input_layer_size + 1)))
theta_prime_2 = np.reshape(fmin.x[hidden_layer_size * (input_layer_size + 1):], (num_labels, (hidden_layer_size + 1)))

In [19]:
theta_prime_2.shape

(10, 26)

In [27]:
def predict(theta1, theta2, X):
    _, _, _, _, h = forward_propagate(X, theta1, theta2)
    y_pred = np.argmax(h, axis=1) + 1
    return y_pred

y_pred = predict(theta_prime_1, theta_prime_2, X)
accuracy = np.sum((y_pred == y))/ len(y_pred)
print("NN accuracy: ", accuracy)

NN accuracy:  0.801
